# define contextual files and DataFrames

In [ ]:
from pandas import read_csv, DataFrame

ave_total_df = read_csv("metabolomics data/day_averaged_concentrations.csv")
ave_total_df.reset_index()
ave_total_df.index = [f'{row["diet"]}_{row["Rec_day_adj"]}' for index, row in ave_total_df.iterrows()]
ave_total_df.drop(["Rec_day_adj", "diet"], axis=1, inplace=True)
ave_total_df.drop([i for i in ave_total_df.index if "WD" in i or "PBS" in i], axis=0, inplace=True)

trials = list(ave_total_df.index)
print(trials)

unique_rec_day = list(map(float, {i.split("_")[1] for i in ave_total_df.index}))  # ['-3.0' '0.0' '11.0' '14.0' '28.0' '3.0' '5.0' '7.0']
print(unique_rec_day, type(unique_rec_day[0]))
def closest_number(number):
    closest = unique_rec_day[0]
    diff_val = abs(number-closest)
    for num in unique_rec_day:
        if abs(number-num) < diff_val:
            closest = num
            diff_val = abs(number-num)
        # elif abs(number-num) == diff_val:  print(closest, num)
    return closest

mega_metadata = read_csv("metabolomics data/a1_meta_c1-3.csv").set_index("seqID")
seqID_recDayAdj = {}
for index, row in mega_metadata.iterrows():
    if row["Rec_day_adj"] > 28 or float(row["Rec_day_adj"]) < -3:   continue
    seqID_recDayAdj[index] = f'{row["Treatment"]}_{closest_number(float(row["Rec_day_adj"]))}'
    # print(index, seqID_recDayAdj[index])
# print(mega_metadata["seqID"].to_list(), mega_metadata["Rec_day_adj"].to_list())
# seqID_recDayAdj = dict(zip([mega_metadata["seqID"].to_list(), mega_metadata["Rec_day_adj"].to_list()]))

print(mega_metadata["Rec_day_adj"].unique())
print({e.split("_")[1] for e in list(seqID_recDayAdj.values())})
display(mega_metadata)

# load and process the OTU file

In [ ]:
# asv_abundances = asv_abundances if "asv_abundances" in locals() else read_csv("metabolomics data/merged_otu_table.txt", sep="\t").set_index("taxonID")
asv_abundances = read_csv("metabolomics data/merged_otu_table.txt", sep="\t").set_index("taxonID")
# display(asv_abundances.iloc[:20])
# asv_abundances.rename(columns=col_names, inplace=True)
# print(list(seqID_recDayAdj.keys()))
asv_abundances = asv_abundances.rename(columns=dict(seqID_recDayAdj)).drop([col for col in asv_abundances.columns if col not in list(seqID_recDayAdj.keys())], axis=1)
display(asv_abundances.iloc[:20])
# print([col for col in asv_abundances.columns if "MMF" in col])

# Define the 

In [ ]:
averaged_asv_vals = asv_abundances.groupby(by=asv_abundances.columns, axis=1).mean().sort_index(axis=1)
averaged_asv_vals.to_csv("metabolomics data/averaged_asv_values.csv")
averaged_asv_vals.drop([col for col in averaged_asv_vals.columns if col not in ave_total_df.index], axis=1, inplace=True)

print(averaged_asv_vals.columns)
display(averaged_asv_vals.iloc[:20])


averaged_asv_vals = read_csv("metabolomics data/averaged_asv_values.csv").set_index("taxonID")
transposed_averaged_asv_vals = averaged_asv_vals.T
trials = list(ave_total_df.index)
print(trials)

df_data= {}
for i, day_diet in enumerate(transposed_averaged_asv_vals.index):
    if day_diet not in trials or day_diet == 'RC-ABX_28.0': continue
    print(day_diet)
    trial_index = trials.index(day_diet)
    trial = day_diet.split("_")
    if trial[0] in trials[trial_index+1]:   future_trial = trials[trial_index+1]
    else:   continue

    trial_name = trial[0]+f"_{(float(trial[1])+float(future_trial.split('_')[1]))/2}"
    df_data[trial_name] = {column: (transposed_averaged_asv_vals.loc[future_trial].values[index]+value)/2
                           for index, (column, value) in enumerate(transposed_averaged_asv_vals.loc[day_diet].items())}

averaged_asv_vals_interday = DataFrame(df_data)
display(averaged_asv_vals_interday)

from json import dump
asv_dict = averaged_asv_vals_interday.to_dict(orient='index')
with open("metabolomics data/averaged_asv_abundances_interday_mapping.json", 'w') as jsonOut:
    dump(asv_dict, jsonOut, indent=3)